In [1]:
#install scispacy
#!pip install scispacy
#!conda activate scispacy

In [ ]:
#create enviorment
#!conda create -n scispacy python=3.6

In [ ]:
#!pip install en_ner_bionlp13cg_md
#!pip install en_core_sci_md

In [ ]:
#activate enviorment
#!conda activate scispacy

In [2]:
import scispacy
import spacy
import en_core_sci_md
import en_ner_bionlp13cg_md
from joblib import dump, load
import os
import numpy as np
import pandas as pd

In [5]:

#open df from the dataprocessing notebook
#os.chdir()
df=pd.read_csv('CORD19_processed.csv', encoding='ISO-8859-1')



In [7]:
#load drug/gene dictionaries from previous notebook.
drug_dic = load('drug_dic.joblib') 
gene_dic = load('gene_dic.joblib') 

In [8]:
#load the nlp trained model
nlp = spacy.load("en_core_sci_md")

In [9]:
def get_entities(document): 
    
#This function gets scientific entites from a str document and the nlp model en_core_sci_md

   
    doc = nlp(document)
    ent_lst=[]
    for ent in doc.ents:
        ent_lst.append(ent)         
    return  ent_lst

In [10]:
#load the nlp trained model
bionlp = spacy.load("en_ner_bionlp13cg_md")

In [11]:
#v.2 checker

def get_genes(document): 
    
#This function gets genes, drugs and organism from a str document and an nlp en_ner_bionlp13cg_md

   
    doc = bionlp(document)
    gene_lst = {}
    drug_lst = {}
    org_lst = {}
    
    for ent in doc.ents:
        if ent.label_ =='GENE_OR_GENE_PRODUCT':
            for k,v in gene_dic.items():
                if (ent.text).lower() in v:
                    gene_lst[ent]= k 
         

        elif ent.label_ =='SIMPLE_CHEMICAL':
            for k,v in drug_dic.items():
                if (ent.text).lower() in v:
                    drug_lst[ent]=k

        elif ent.label_ =='CELL' or ent.label_ =='ORGANISM' or ent.label_ =='ORGANISM_SUBSTANCE' or ent.label_ =='TISSUE':
            org_lst[ent]=ent.text
    return  gene_lst,drug_lst,org_lst 

#returns list of dicts: 
#[0] dictionary with gene_lst
#[1] dictionary with drug_lst
#[2] dictionary with organism/cell entities for searching propuses

In [12]:
def get_entiteis_cmb (srt):
#Funciton to combine get_entities() and get_genes from a document into a list of lists
    all_lst = []
    ent = get_entities(srt)
    bio = get_genes(srt)
    all_lst.append(ent)
    for i in range(len(bio)):
        all_lst.append(bio[i])
    return all_lst

In [13]:
#This function aims to detect a potential gene regulation by looking on spans of +/-3 scispacy entities 
#from a detected gene. The method looks for terms commonly used to express up or down gene regulation.
#Future steps will focus in improving the 'regulation' identificaion. 


gene_up = ['up', 'increase', 'increased', 'up-regulation', 
           'upregulation', 'overexpression','activation','overexpressed','overexpressing',
           'induction', 'high', 'activated',
           'activated', 'activating', 'amplified','enriched', 'insensitive',
           'enhanced' ,'enhancing'   
]


gene_down = ['inhibited', 'inhibiting','repressing', 'suppressing', 'capping',
             'degrading', 'derepressing', 'down', 'decrease', 'decreasing','down-regulation',
             'downregulation', 'repression', 'repressed','repress','decrease', 'decreassing',              
            ]

def get_regulation(lst_lsts):
    
    #function that creates spans +/- 3  of entity type == genes and checks for terms used to describe
    #gene up/down-regulation. If found, new class (up/down) assigned.
    
    ent_lst = lst_lsts[0]
    gn_lst = lst_lsts[1]
    span_gn = []

    for i in range(len(ent_lst)):
        for k,v in gene_dic.items():
            if (ent_lst[i].text).lower() in v:
                    span_gn.append(ent_lst[i-3:i+3])

    span_gn = list(map(str, span_gn))
    #print(span_gn)
    g_span = dict(list(zip(gn_lst,span_gn)))
    #print(g_span)
    #print(g_span)
    gn_up = []
    gn_dw = []
    gn_n = []
    
    
    for k,v in g_span.items():
        
        if any(elem.lower() in v for elem in gene_up):
            gn_up.append(k)
            #print(k)
        elif any(elem.lower() in v for elem in gene_down):
            gn_dw.append(k)
        else:
            gn_n.append(k)

    
    #returns list of lists with genes_up at [0], genes_down at [1] and genes_non_mentioned at [2]  
    return gn_up,gn_dw,gn_n


    

In [14]:
#This funtion will get a row from a dataframe and perform the scispacy and span collection from the corpus column.
#At its current form it takes 3 seconds per row to compute. 

def process_row(row):
    """returns a row with some extra column"""
    scispacy = get_entiteis_cmb(row['corpus'])
    spans = get_regulation(scispacy)

    ent_text = []
    for ent in scispacy[0]:      
        ent_text.append(ent.text)
    a = list(scispacy[1].values())
    genes = []
    genes = genes + list(scispacy[1].values())
    row['Genes'] = genes
    
    drugs = []
    drugs = drugs + list(scispacy[2].values())
    row['Drugs'] = drugs
    
    row['Biomedical_Entities'] = ent_text
    row['Genes Upregulated'] = spans[0]
    row['Genes Down_regulated'] = spans[1]
    row['Genes Non_regulated'] = spans[2]

    return row



In [15]:
#This will take several days to run, it might be advisabel to run it in small batches
#and then marge the df toghether
df_mesminer = df.apply(process_row, axis=1)

df_mesminer



,Old_index,cord_uid,publish_time,preprint,corpus,Genes,Drugs,Biomedical_Entities,Genes Upregulated,Genes Down_regulated,Genes Non_regulated
0,4203,p58n51xv,05/12/2019,Peer-Review,Crandell-Rees Feline Kidney Cell (CRFK) immort...,"[s100a4, vim, vwf, s100a4]",[],"[Crandell-Rees Feline Kidney Cell, CRFK, immor...",[],[],"[(S100A4), (vimentin), (vWF), (S100A4)]"
1,4213,heo6mtxk,05/12/2019,Peer-Review,African swine fever transmissible animal consu...,[],[],"[African swine fever, transmissible, animal co...",[],[],[]
2,4227,4tt0vnr4,02/12/2019,Peer-Review,Resident memory cell (T(RM) cells) important f...,"[cxcr6, cd8a, cxcr6, cxcl16, cxcr6, cxcl16, cd...",[],"[Resident memory cell, T(RM, first-line, defen...",[],"[(CD8), (CXCR6), (CXCR6), (CXCL16)]","[(CXCR6), (CXCL16)]"
3,4230,9c0zrp7p,03/12/2019,Peer-Review,"infectious outbreak emerge, public agency ofte...",[],[],"[infectious outbreak, public agency, vaccinati...",[],[],[]
4,4232,iaktm72a,03/12/2019,Peer-Review,work present simple one-pot protocol achieve c...,[],"[curcumin, sodium citrate, curcumin, curcumin,...","[one-pot protocol, core-doped shell, nanohybri...",[],[],[]


In [ ]:
#export the resutls for the next step
df_mesminer.to_csv('df_mesminer.csv',encoding='utf-8')
